# `run_201905220848`

## Configuration

### `EstNLTK` version
```
(py36) paul@hp:~/Projects/cda-data-cleaning$ git -C ../estnltk show --summary
commit fc9f4ae52ff7e9c24145fe9b31dbcec161f2b10a (HEAD -> devel_1.6, origin/devel_1.6)
Author: pault <pault@ut.ee>
Date:   Thu May 23 14:06:58 2019 +0300

    new docstrings in splitting.py
```

### `cda-data-cleaning` version
```
(py36) paul@hp:~/Projects/cda-data-cleaning$ git show --summary
commit 96bc8456c87b23e709c5a6d3e449d1a443a4ca5a (HEAD -> master, origin/master, origin/HEAD)
Author: pault <pault@ut.ee>
Date:   Thu May 23 22:05:35 2019 +0300

    #1128 add BoneDensityTagger and WeightChangeTagger into MeasurementTagger

```

### `egcut_epi_original.ini`
```
(py36) ptammo@p12:~/Projects/cda-data-cleaning$ cat configurations/egcut_epi_original.ini 
[database-configuration]
host = p12.stacc.ee
port = 5432
database_name = egcut_epi

username = ptammo
password = xxxxxx

role = egcut_epi_work_create
table_create_role = egcut_epi_work_create
table_read_role = egcut_epi_work_read

original_schema = original
work_schema = work


[luigi]
folder = luigi_targets


# LEGACY TO BE REMOVED
[classifications-database-configuration]
host = p12.stacc.ee
port = 5432
database_name = qs_rel
schema = rel_classifications
```

## `CreateTextsCollection`
```bash
luigi --scheduler-port 8089 --module cda_data_cleaning CreateTextsCollection  --prefix=run_201905220848 --conf=egcut_epi_original.ini --workers=8
```

## `CreateEventsCollection`
```bash
luigi --scheduler-port 8089 --module cda_data_cleaning CreateEventsCollection --prefix=run_201905220848 --conf=egcut_epi_original.ini --workers=1
```

## `TagMeasurements`
```bash
luigi --scheduler-port 8089 --module cda_data_cleaning TagMeasurements --prefix=run_201905220848 --conf=egcut_epi_original.ini --workers=1 --log-level=INFO
```

## `TagMeasurementsDev`
```bash
luigi --scheduler-port 8089 --module cda_data_cleaning.measurement_extraction.development TagMeasurementsDev --prefix=run_201905220848 --conf=egcut_epi_original.ini --workers=1 --log-level=INFO
```

## Run `ExportMeasurementsToTable`
```bash
luigi --scheduler-port 8089 --module cda_data_cleaning ExportMeasurementsToTable --prefix=run_201905220848 --conf=egcut_epi_original.ini --workers=1 --log-level=INFO
```

This creates `egcut_epi.work.run_201905220848_extracted_measurements` table with 1,408,408 rows.

## `TagMeasurementsFlatDiff`
```bash
luigi --scheduler-port 8089 --module cda_data_cleaning.measurement_extraction.development TagMeasurementsFlatDiff --prefix=run_201905220848 --conf=egcut_epi_original.ini --workers=1 --log-level=INFO
```

## Results

In [1]:
from estnltk.storage import PostgresStorage

storage = PostgresStorage(pgpass_file='~/.pgpass',
                          dbname='egcut_epi',
                          schema='work',
                          role='egcut_epi_work_create')

INFO:storage.py:41: connecting to host: 'p12.stacc.ee', port: '5432', dbname: 'egcut_epi', user: 'ptammo'
INFO:storage.py:57: schema: 'work', temporary: False, role: 'egcut_epi_work_create'


### `run_201905220848_texts` collection

In [2]:
PREFIX = 'run_201905220848'

collection = storage[PREFIX + '_texts']
collection

In [3]:
collection.selected_layers = ['anonymised', 'event_tokens', 'event_headers', 'event_segments']

In [4]:
# find a text with an interesting `event_segments` layer
stop = False
for t in collection:
    if len(t.event_segments) > 1:
        for span in t.event_segments:
            if span.DOCTOR:
                stop = True
                break
    if stop:
        break

In [5]:
t.event_segments

Layer(name='event_segments', attributes=('header', 'header_offset', 'DATE', 'HEADERWORD', 'DOCTOR', 'DOCTOR_CODE', 'SPECIALTY', 'SPECIALTY_CODE', 'ANONYM'), spans=SL[Span(start=0, end=26, text=' UURINGUD JA PROTSEDUURID '),
Span(start=95, end=305, text='Radioloogiline uuring, vastus nr 1262_15163_1\n\nKIRJELDUS: Kopsud infiltr. koldeleiuta, kopsujoonis struktuurne, hiilused pole laienenud. Süda, mediatsiinum on ii. Diafragma tavakõrgusel, later. siinused vabad. '),
Span(start=318, end=1212, text='WBC 9.16 (3,5 .. 8,8 E9/L )  \nRBC 4.99 (4,2 .. 5,7 E12/L )  \nHGB 145 (134 .. 170 g/L )  \nHCT 44 (40 .. 50 % )  \nMCV 88.0 (82 .. 98 fL )  \nMCH 29.1 (27 .. 33 pg )  \nMCHC 330 (317 .. 357 g/L )  \nPLT 270 (145 .. 390 E9/L )  \nRDW-CV 13.5 (11,6 .. 14 % \n\nB-HbA1c 6.26 (4.50 .. 5.70 % kogu Hb-st )  \nS,P-Glükoos 6.4 (mmol/L ) \nS,P-CRP &lt;1 (&lt;5 ) \nS,P-Na 140 (136 .. 145 mmol/L ) \nS,P-K 3.8 (3.5 .. 5.1 mmol/L ) \nS,P-ASAT 32 (&lt;38 U/L ) \nS,P-ALAT 42 (&lt;41 U/L ) \nS,P-Kusihape 435 (&lt;500 µmol/L ) \nS,P-Kolesterool 5.0 (3.9 .. 7.8 mmol/L ) \nfS,fP-HDL-Chol 1.00 (0.80 .. 2.10 mmol/L ) \nfS,fP-LDL-Chol 3.52 (2.00 .. 5.30 mmol/L ) \nfS,fP-Triglütseriidid 1.66 (0.45 .. 2.60 mmol/L ) \nS,P-CK-MBm 15.86 (&lt;6.73 ng/mL ) \nS,P-Troponiin T 0.011 (&lt;0.030 ng/mL ) \nGlomerulaarfiltratsiooni kiirus (eGFR) 99 (&gt;90 mL/min/1.73m2 ) \nS,P-Kreatiniin 73 (62 .. 106 µmol/L ) \nPatsiendi vanus 68 (aastat ) '),
Span(start=1284, end=2526, text="Radioloogiline uuring, vastus nr 1167B_15164_1\n\nKIRJELDUS: SKG teostatud 6F sondidega paremalt transfemoraalselt res. dr. Lindlaaneнe poolt, protseduur tüsistusteta, probleemideta.AR 118/60 Kaasuvalt hüpertoonik, diabeetik, suitsetab. ARVAMUS: SKG valem: B N Angiograafiliselt balansseeritud verevarustustüüp. Pärgarterites ahenemisi ei ol \xa0 07.07.2009 08:41 - HANSEN, SIRJE - D05395 - E09 - kardioloogia\nTÄISMAHUS EHHOKARDIOGRAAFIA, 07.07.2009; teostaja: MUDA, PIIBE - D04433\nNähtavus suboptimaalne. Siinusrütm. Vasaku vatsakese kerge kontsentriline hüpertroofia. Üldine süstoolne funktsioon normis, EF 56%. Diastoolse funktsiooni kerge häire mitraalvoolu ja koe-Doppleri järgi (E/E' = 9 lateraalsel, E/E' = 107 septaalsel). Kindlat segmentaarse kontraktiilsuse häiret suboptimaalse nähtavuse foonil ei ole. Parema vatsakese õõne suurus ja üldine süstoolne funktsioon normis. Diastoolse funktsiooni kerge häire koe-Doppleri järgi. Aordiklapp 3-hõlmne, servad fibrootilised, minimaalne regurgitatsioon. Pulmonaalklapi funktsioon normis. Atrioventrikulaarklapid iseärasusteta, funktsioonid normis (minimaalsed regurgitatsioonid). Kojad norm. suurusega: vasak koda 3,3 x 4,9 x 3,3 cm, parem koda 4,8 x 3,1 cm. Periakrdiõõnes vedelikku ei ole. \n")])

### `run_201905220848_events` collection

In [6]:
collection = storage[PREFIX + '_events']
collection

,data type
texts_id,integer
epi_id,text
epi_type,text
schema,text
table,text
field,text
row_id,text
effective_time,timestamp without time zone
header,text
header_offset,integer


In [7]:
for id_, text, meta in collection.select(
    collection_meta=['texts_id', 'epi_id', 'epi_type', 'schema', 'table', 'field',
                     'row_id', 'effective_time', 'header', 'header_offset', 'event_offset'],
    layers=['anonymised']):

    if len(text.anonymised) > 1:
        break

In [8]:
meta

{'texts_id': 11163,
 'epi_id': '10428973',
 'epi_type': 's',
 'schema': 'original',
 'table': 'anamnesis',
 'field': 'anamsum',
 'row_id': '1109',
 'effective_time': None,
 'header': None,
 'header_offset': None,
 'event_offset': 0}

In [9]:
text

Text(text='Patsient hospitaliseeritud plaanilise haigena uuringuteks suunatuna Pärnust dr. <ANONYM id="0" type="per" morph="_H_ sg n"/>\'e poolt ventrikulaarse ekstrasüstooliaga. Juhuleiuna tervisekontrollis avastati VES-ia, patsient seda ise väga ei tunneta, seega ei ole teada, mis ajast probleem tekkinud on. Pärnus tehtud Holter-monitooringul\xa0siinusrütmi foonil 8% VES-eid ööpäeva jooksul, esinemine ei sõltu koormusest. Alustati ravi Sotahexal 80mg päevas, kuid patsient seda üle 2 nädala kasutada ei saanud kuna enesetunne pigem halvenes, tekkis jõuetustunne, võimalik et hüpotoonia ning patsient lõpetas ravimi juba augustikuus. Pärnu Haiglas tehtud ehhokardiograafilisel uuringul kirjeldatakse kodade kerget dilatatsiooni ja vasaku vatsakese II astme diastoolse lõõgastumise häiret. Kuna anamneesis sagedased viirushaiguste põdemised ning tonsilliidid, siis kahtlus tonsillogeensele kardiomüopaatiale, LOR konsultatsioon Pärnus juba tehtud ja planeeritud tonsillektoomia 2011.a. jaanuarikuusse.Samuti\xa0olnud 2010.a. algul uroplasma infektsioon, sai AB ravi. 2009.a. põdenud püelonefriiti.\xa0Eluanamnees: Töötab politseinikuna, tegeleb füüsilise treeninguga, käib jõusaalis. Ei suitseta.\nEelnev ravi: Sotahexal 40mgx2 juulikuus ~2 nädalat, Panangin 1x2.\nHaige saabudes heas seisundis, oluliste subjektiivsete kaebusteta. Objektiivsel vaatlusel tonsillid väikesed, "näritud" välimusega, näha mandlikorke. <ANONYM id="1" type="per" morph="_S_ sg ad"/> kõrvalkahinaid ei kuule, auskultatsioonil minutis 1-4 ekstrasüstolit, fr. 70x\'. Vererõhk normaalne 110/75 mmHg.Teostatud dünaamika hindamiseks uus Holter monitooring, kus vaatlusalusel perioodil siinusrütm keskmise sagedusega 64x\', selle foonil esines 12% monomorfsed VES-id, esines üksikuid bi- ja trigeminusepisoode ja 1 paariline VES. MRT uuringul südamest vasaku vatsakese müokardi\xa0kahjustusele s.h. müokardiidile\xa0viitavat leidu ei sedasta, samuti on normaalsed kõik südame õõnte mõõdud.Teostatud ka koormustest, kus\xa0väga hea koormustaluvus 15,5 MET,\xa0esineb harv VES rahuolekus, koormusel ja taastumisel; monomorfne keskmiselt 4 VES-it minutis, koormuse II poolel frekventsi kiirenedes VES-ia harvem. Vereanalüüsides kõrvalekaldeid normist ei sedasta.Patsient ei kuulu ablatsioonravile, ega vaja ka medikamentoosset rütmihäirete ravi. Esialgu vajalik ja juba eelnevalt planeeritud tonsillektoomia, seejärel ~2 kuu möödudes teha ambulatoorselt Pärnus korduvalt Holter-monitooring. Kuna ekstrasüstoolia tekkepõhjus -ja aeg teadmata, siis on vastunäidustatud on suur jõutreening, sobivad isotoonilised harjutused (jooks, kõnd, ujumine jms.) mõõdukas tempos.\n')

## `measurements_flat_diff` layer

Compare `measurements` and `measurements_dev` layer.

In [10]:
from estnltk.taggers.standard_taggers.diff_tagger import DiffSampler

diff_sampler = DiffSampler(collection=collection, layer='measurements_flat_diff')

In [11]:
diff_sampler.layer_meta.sum()

id                     4374078939856
text_id                4374075982128
modified_spans         0            
missing_spans          0            
extra_spans            7885         
extra_annotations      7885         
missing_annotations    0            
overlapped             0            
prolonged              0            
shortened              0            
dtype: int64

In [12]:
for id_, _, span in diff_sampler.sample_spans(20, 'extra_spans'):
    span.display()

text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
T-skoor 0.5,105,116,extra,measurements_dev,MEASUREMENT,T-skoor,0.5,,,,,LUUTIHEDUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
"T-skoor -2,4",1454,1466,extra,measurements_dev,MEASUREMENT,T-skoor,-2.4,,,,,LUUTIHEDUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
juurde 8 kg,1055,1067,extra,measurements_dev,MEASUREMENT,juurde,8,,,,,KAALUTÕUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
kaotanud ca 20 kg,1135,1153,extra,measurements_dev,MEASUREMENT,kaotanud ca,20,,,,,KAALULANGUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
suure söögiisu tõttu 5 kg,41,67,extra,measurements_dev,MEASUREMENT,suure söögiisu tõttu,5,,,,,KAALUTÕUS


...

text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
juurde võtnud 6-7kg,772,792,extra,measurements_dev,MEASUREMENT,juurde võtnud 6,7,,,,,KAALUTÕUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
tõusis kehakaal 10 kg,80,102,extra,measurements_dev,MEASUREMENT,tõusis kehakaal,10,,,,,KAALUTÕUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
5 kg alla,237,247,extra,measurements_dev,MEASUREMENT,alla,5,,,,,KAALULANGUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
tõusnud ravi käigus 8 kg,154,179,extra,measurements_dev,MEASUREMENT,tõusnud ravi käigus,8,,,,,KAALUTÕUS


text,start,end,span_status,input_layer_name,name,OBJECT,VALUE,UNIT,MIN,MAX,DATE,REGEX_TYPE
8 kg alla,85,95,extra,measurements_dev,MEASUREMENT,alla,8,,,,,KAALULANGUS


In [13]:
storage.close()